In [126]:
# Importing required packages
import numpy as np
from gurobipy import *
import pandas as pd
import os

### <font color='red'> Location of the file</font>

In [127]:
os.chdir(r'C:\Users\said_\Google Drive\THE CITY COLLEGE OF NEW YORK\Project of Puerto Rico\CODES JUPYTER\Water&Power\Power-Network-Model-main\Power-Network-Model-main')

In [40]:
# ---------------------------------Setting code input parameters (for both current and future grid) --------------
# Setting the list of link indices which fail. Refere to links tab of input data excel files for indices
power_failed_links = list(range(10))

# Setting the list of failure times (one for each failed link)
power_fail_times = [0] * 10

# Setting the number of available repair teams
power_num_servers = 1

# Setting the lenght of planning horizon
num_time_periods = 10

# --------------------------------------------------------------------------------------------------------------------

In [41]:
# ----------------------------- Reading network input data for the current grid --------------------------------------
# Setting the excel input file
import_current_data = 'Data/current_grid.xls'

# Reading the nodes data frame from the input file
current_power_nodes = pd.read_excel(import_current_data, sheet_name=0, header=0, index_col=0)

# Reading links data frame from the input file
current_power_links = pd.read_excel(import_current_data, sheet_name=1, header=0, index_col=0)

# Setting the number of nodes using input data
current_power_num_nodes = current_power_nodes.shape[0]

# Setting the number of links using input data
current_power_num_links = current_power_links.shape[0]

# ---------------------------------------------------------------------------------------------------------------------

In [42]:
# Creating empty dictionaries to store the input parameters to the network flow model (current grid)
current_power_demand = {}
current_power_generation_capacity = {}
current_power_storage_capacity = {}

# Using input data to fill in demand and generation capacity dictionaries (current grid)
for i in range(current_power_num_nodes):
    current_power_demand[i] = current_power_nodes["Demand"].iloc[i]
    current_power_generation_capacity[i] = current_power_nodes["Generation_Capacity"].iloc[i]

# Filling in the repair duration list (current grid)
current_power_repair_durations = []
for i in range(current_power_num_links):
    current_power_repair_durations.append(current_power_links["Repair_Duration"].iloc[i])

# Forming the set of repair teams (for both current and future grids)
power_servers = range(power_num_servers)

# Forming the set of time periods (for both current and future grids)
time_periods = range(num_time_periods)

# Creating the model object for network flow model (current grid)
current_power_res_model = Model()

In [43]:
# -----------  Defining the decision variables for optimal recovery of the current grid -------------------------

# y_itj is a binary variable and is equal to one if j-th repair team starts to fix i-th link at time period t
current_power_y = current_power_res_model.addVars(((i, t, j) for i in range(current_power_num_links)
                                                   for t in time_periods for j in power_servers), vtype=GRB.BINARY)

# s_it is a binary deicsion variable and is equal to one if i-th link is functional at time period t
current_power_s_links = current_power_res_model.addVars(((i, t) for i in range(current_power_num_links)
                                                         for t in time_periods), vtype=GRB.BINARY)

# ps_it is a continuous decision variable and is equal to the power supplied at i-th node at time period t
current_power_supplied = current_power_res_model.addVars((i, t) for i in range(current_power_num_nodes)
                                                         for t in time_periods)

# pg_it is a continuous decision variable and is equal to the power generated at i-th node at time period t
current_power_generated = current_power_res_model.addVars((i, t) for i in range(current_power_num_nodes)
                                                          for t in time_periods)
# f_ijt is a continuous decision variable and is equal to the power transmitted from i-th to j-th node at time period t
current_power_transmitted = current_power_res_model.addVars((i, j, t) for i in range(current_power_num_nodes)
                                                    for j in range(current_power_num_nodes) for t in time_periods)

# ---------------------------------------------------------------------------------------------------------------------

In [44]:
# --------------- Using input data to form power transmission capacity matrix for current grid  ---------------------

# Creating the matrix with all it its elements equal to zero
current_power_transmission_capacity = np.zeros((current_power_num_nodes, current_power_num_nodes))

# Reading source and target nodes of the different links using input data
current_power_link_sources = current_power_links["From_Node"].values
current_power_link_targets = current_power_links["To_Node"].values

# Reading links transmission capacities from the input data
current_power_link_capacities = current_power_links["Transmission_Capacity"].values

# Reading links type from the input data, type 1 links are one-way links while type 2 links are two-way ones
current_power_link_types = current_power_links["Type"].values

# Using the above arrays to fill in the non-zero values in the tranmission capacity matrix created above
for i in range(current_power_num_links):
    current_power_transmission_capacity[current_power_link_sources[i], current_power_link_targets[i]] = \
        current_power_link_capacities[i]
    if current_power_link_types[i] == 2:
        current_power_transmission_capacity[current_power_link_targets[i], current_power_link_sources[i]] = \
            current_power_link_capacities[i]

# -----------------------------------------------------------------------------------------------------------------

In [45]:
# Creating the empty dictionary for network links. This dictionaty, takes (i,j) pairs of nodes as input and returns the
# index of the link between the nodes i and j of the network (current grid)
current_power_links_dic = {}

# Filling in the key-value pairs in the dictionary created above (current grid)
for i in range(current_power_num_links):
    current_power_links_dic[(current_power_link_sources[i], current_power_link_targets[i])] = i
    if current_power_link_types[i] == 2:
        current_power_links_dic[(current_power_link_targets[i], current_power_link_sources[i],)] = i

# Setting power generation capacity for the current power network
current_power_generation_capacity = current_power_nodes["Generation_Capacity"].values

In [46]:
# ------------------------------------- Adding constrains to the optimal recovery problem for the current grid --------
# These constraints state that after failure time each link of the power network will not be functional before
# it is fixed
current_power_res_model.addConstrs(current_power_s_links[power_failed_links[i], t] <=
                                   quicksum(current_power_y[power_failed_links[i], u, j] for u in
                                            range(int(power_fail_times[i]), t - int(
                                                current_power_repair_durations[power_failed_links[i]] + 1))
                                            for j in power_servers) for i in range(len(power_failed_links))
                                   for t in range(power_fail_times[i], num_time_periods))

# These constraints set the functionality indicator to one for all links of the power network which do not fail
current_power_res_model.addConstrs(current_power_s_links[i, t] == 1 for i in range(current_power_num_links)
                                   for t in time_periods if i not in power_failed_links)

# These constraints set the functionality indicator to one for all failed links of the power network
# before their failure
current_power_res_model.addConstrs(current_power_s_links[power_failed_links[i], t] == 1
                                   for i in range(len(power_failed_links)) for t in range(power_fail_times[i]))

# These constraints mean that once a repair team starts fixing a power network link, it will not be available until
# it finishes the ongoing restoration task
current_power_res_model.addConstrs(current_power_y[i, j, k] <= 1 - current_power_y[p, q, k] for i in
                                   range(current_power_num_links) for j in time_periods for k in power_servers
                                   for p in range(current_power_num_links) if p != i
                                   for q in range(j, min(j+int(current_power_repair_durations[i]), num_time_periods)))

# Power balance equation for different nodes of the network
current_power_res_model.addConstrs(quicksum(current_power_transmitted[k, i, t] for k in range(current_power_num_nodes))
                                   + current_power_generated[i, t] == current_power_supplied[i, t] +
                                   quicksum(current_power_transmitted[i, k, t]
                                            for k in range(current_power_num_nodes))
                                   for i in range(current_power_num_nodes) for t in range(num_time_periods))

# Power generation capacity for the current power network
current_power_res_model.addConstrs(current_power_generated[i, t] <= current_power_generation_capacity[i]
                                   for i in range(current_power_num_nodes) for t in time_periods)

# Power transmission capacity for links considering only links availability
for i in range(current_power_num_nodes):
    for j in range(current_power_num_nodes):
        if current_power_transmission_capacity[i, j] > 0:
            current_power_res_model.addConstrs(current_power_transmitted[i, j, t] <=
                                               current_power_transmission_capacity[i, j] *
                                               current_power_s_links[current_power_links_dic[(i, j)], t]
                                               for t in time_periods)
        else:
            current_power_res_model.addConstrs(current_power_transmitted[i, j, t] == 0 for t in time_periods)

# Supply limit at each node
current_power_res_model.addConstrs(current_power_supplied[i, t] <= (current_power_demand[i])
                                   for i in range(current_power_num_nodes) for t in range(num_time_periods))
# ---------------------------------------------------------------------------------------------------------------------

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,


In [47]:
# Forming the objective Function for the current power network
current_power_res_model.setObjective(quicksum((current_power_demand[i] - current_power_supplied[i, t])
                                              for i in range(current_power_num_nodes) for t in time_periods),
                                     GRB.MINIMIZE)

# Solving the power restoration model for the current power network
current_power_res_model.update
current_power_res_model.optimize()

# Extracting the optimal solution for the current power network
current_power_supplied = np.reshape(current_power_res_model.getAttr('x', current_power_supplied).values(),
                            (current_power_num_nodes, num_time_periods))
current_power_avail_mat = np.reshape(current_power_res_model.getAttr('x', current_power_s_links).values(),
                             (current_power_num_links, num_time_periods))
current_power_recovery_times = []
for cnt, val in enumerate(power_failed_links):
    current_power_s_component = current_power_avail_mat[val, power_fail_times[cnt]:]
    current_power_recovery_times.append(next((i for i, x in enumerate(current_power_s_component) if x), None))

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 919126 rows, 196700 columns and 2023911 nonzeros
Model fingerprint: 0xa45e2d40
Variable types: 193200 continuous, 3500 integer (3500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 25700.000000
Presolve removed 918796 rows and 196116 columns
Presolve time: 0.58s
Presolved: 330 rows, 584 columns, 1665 nonzeros
Found heuristic solution: objective 23900.000000
Variable types: 476 continuous, 108 integer (108 binary)

Root relaxation: objective 9.577631e+03, 393 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9577.63095    0   17 23900.0000 9577.63095  59.9%     -    0s
H    0     0                    10990.000000 9577.63

In [48]:
current_power_recovery_times

[None, 9, 7, 4, None, None, None, 3, None, None]

In [49]:
# ----------------------------- Reading network data for the future grid ---------------------------------------------
# Setting the input data file
import_new_data = 'Data/future_grid.xls'

# Forming nodes and links data frames for the future grid using input data file
future_power_nodes = pd.read_excel(import_new_data, sheet_name=0, header=0, index_col=0)
future_power_links = pd.read_excel(import_new_data, sheet_name=1, header=0, index_col=0)

# Setting the number of nodes and links
future_power_num_nodes = future_power_nodes.shape[0]
future_power_num_links = future_power_links.shape[0]
# --------------------------------------------------------------------------------------------------------------------

In [50]:
# -------------------------------- Creating empty dictionaries to store the input parameters of the network flow model
# for the future grid
future_power_demand = {}
future_power_generation_capacity = {}
future_power_storage_capacity = {}

# Using input data to fill in demand, generation capacity and storage capacities
for i in range(future_power_num_nodes):
    future_power_demand[i] = future_power_nodes["Demand"].iloc[i]
    future_power_generation_capacity[i] = future_power_nodes["Generation_Capacity"].iloc[i]
    future_power_storage_capacity[i] = future_power_nodes['Storage_Capacity'].iloc[i]

# Using input data to fill in the repair duration list
future_power_repair_durations = []
for i in range(future_power_num_links):
    future_power_repair_durations.append(future_power_links["Repair_Duration"].iloc[i])

# Creating network flow model object for the future grid
future_power_res_model = Model()

In [51]:
# ---------------------  Defining the decision variables for the optimal recovery of the future grid -----------------
# y_ijt is a binary decision variable equal to one if j-th team is assigned to i-th link at time period t
future_power_y = future_power_res_model.addVars(((i, t, j) for i in range(future_power_num_links)
                                                   for t in time_periods for j in power_servers), vtype=GRB.BINARY)

# s_it is a binary decision variable equal to one if i-th link is function at time period t
future_power_s_links = future_power_res_model.addVars(((i, t) for i in range(future_power_num_links)
                                                         for t in time_periods), vtype=GRB.BINARY)

# ps_it is a continuous decision variable denoting the power supplied at i-th node of the network at time period t
future_power_supplied = future_power_res_model.addVars((i, t) for i in range(future_power_num_nodes)
                                                         for t in time_periods)

# pg_it is a continuous decision variable noting the power generated at i-th node of the network at time period t
future_power_generated = future_power_res_model.addVars((i, t) for i in range(future_power_num_nodes)
                                                          for t in time_periods)

# pt_ijt is a continuous decison variable noting the power transmitted over the link between i-th and j-th nodes of
# the network
future_power_transmitted = future_power_res_model.addVars((i, j, t) for i in range(future_power_num_nodes)
                                                    for j in range(future_power_num_nodes) for t in time_periods)

# sp_it is a continuous decision variable denoting the power stored at i-th node at time period t
future_power_stored = future_power_res_model.addVars((i, t) for i in range(future_power_num_nodes)for t in time_periods)

# -------------------------------------------------------------------------------------------------------------------

In [52]:
# --------------- Using input data to form power transmission capacity matrix for the future grid ---------------------

# Creating the transmission capacity matrix and setting all its elements to zero
future_power_transmission_capacity = np.zeros((future_power_num_nodes, future_power_num_nodes))

# Reading the source and target nodes of different links of the network from input data
future_power_link_sources = future_power_links["From_Node"].values
future_power_link_targets = future_power_links["To_Node"].values

# Reading transmission capacities using input data
future_power_link_capacities = future_power_links["Transmission_Capacity"].values

# Reading link type using input data. Link type 1 are one-way while link type 2 are two-way links
future_power_link_types = future_power_links["Type"].values

# Filling in the non-zero values of transmission capacity matrix using the above arrays
for i in range(future_power_num_links):
    future_power_transmission_capacity[future_power_link_sources[i], future_power_link_targets[i]] = \
        future_power_link_capacities[i]
    if future_power_link_types[i] == 2:
        future_power_transmission_capacity[future_power_link_targets[i], future_power_link_sources[i]] = \
            future_power_link_capacities[i]

# ---------------------------------------------------------------------------------------------------------------------

In [53]:
# Creating the empty links dictionary.
future_power_links_dic = {}

# Filling in the key-value pairs in the links dictionary defined above. This dictionary takes node pairs (i, j) as
# input and returns the index of the link between the nodes
for i in range(future_power_num_links):
    future_power_links_dic[(future_power_link_sources[i], future_power_link_targets[i])] = i
    if future_power_link_types[i] == 2:
        future_power_links_dic[(future_power_link_targets[i], future_power_link_sources[i], )] = i

# Setting power generation capacity for the future grid
future_power_generation_capacity = future_power_nodes["Generation_Capacity"].values

In [54]:
# --------------------- Adding constraints to optimal recovery model for the future grid -----------------------------
# These constraints state that after failure time each link of the power network will not be functional before
# it is fixed
future_power_res_model.addConstrs(future_power_s_links[power_failed_links[i], t] <=
                               quicksum(future_power_y[power_failed_links[i], u, j] for u in range(
                                   int(power_fail_times[i]), t - int(
                                       future_power_repair_durations[power_failed_links[i]] + 1)) for j in power_servers)
                               for i in range(len(power_failed_links)) for t in range(power_fail_times[i],
                                                                                      num_time_periods))

# These constraints set the functionality indicator to one for all links of the power network which do not fail
future_power_res_model.addConstrs(future_power_s_links[i, t] == 1 for i in range(future_power_num_links)
                                   for t in time_periods if i not in power_failed_links)

# These constraints set the functionality indicator to one for all failed links of the power network before their
# failure
future_power_res_model.addConstrs(future_power_s_links[power_failed_links[i], t] == 1
                                   for i in range(len(power_failed_links)) for t in range(power_fail_times[i]))

# These constraints mean that once a repair team starts fixing a power network link, it will not be available until
# it finishes the ongoing restoration task
future_power_res_model.addConstrs(future_power_y[i, j, k] <= 1 - future_power_y[p, q, k] for i in
                                   range(future_power_num_links) for j in time_periods for k in power_servers
                                   for p in range(future_power_num_links) if p != i
                                   for q in range(j, min(j+int(future_power_repair_durations[i]), num_time_periods)))

# Power balance equation for the new power network at time periods 0, 1, 2, ..., nt-1
future_power_res_model.addConstrs(future_power_stored[i, t] +
                               quicksum(future_power_transmitted[k, i, t] for k in range(future_power_num_nodes)) +
                               future_power_generated[i, t] == future_power_supplied[i, t] +
                                   quicksum(future_power_transmitted[i, k, t] + future_power_stored[i, t+1]
                                            for k in range(future_power_num_nodes))
                                   for i in range(future_power_num_nodes) for t in range(num_time_periods-1))

# Power balance equation for the new power network at the last time period
future_power_res_model.addConstrs(future_power_stored[i, num_time_periods-1] +
                               quicksum(future_power_transmitted[k, i, num_time_periods-1] for k in range(
                                   future_power_num_nodes)) + future_power_generated[i, num_time_periods-1] ==
                                  future_power_supplied[i, num_time_periods-1] +
                                  quicksum(future_power_transmitted[i, k, num_time_periods-1] for k in range(
                                      future_power_num_nodes)) for i in range(future_power_num_nodes))

# Power generation capacity for the new power network
future_power_res_model.addConstrs(future_power_generated[i, t] <= future_power_generation_capacity[i]
                                   for i in range(future_power_num_nodes) for t in time_periods)

# Power storage capacity for the new power network
future_power_res_model.addConstrs(future_power_stored[i, t] <= future_power_storage_capacity[i]
                               for i in range(future_power_num_nodes) for t in time_periods)

# Power transmission capacity for links considering only links availability for the new power network
for i in range(future_power_num_nodes):
    for j in range(future_power_num_nodes):
        if future_power_transmission_capacity[i, j] > 0:
            future_power_res_model.addConstrs(future_power_transmitted[i, j, t] <=
                                               future_power_transmission_capacity[i, j] *
                                               future_power_s_links[future_power_links_dic[(i, j)], t]
                                               for t in time_periods)
        else:
            future_power_res_model.addConstrs(future_power_transmitted[i, j, t] == 0 for t in time_periods)

# Supply limit at each node of the new power network
future_power_res_model.addConstrs(future_power_supplied[i, t] <= (future_power_demand[i])
                                   for i in range(future_power_num_nodes) for t in range(num_time_periods))

# --------------------------------------------------------------------------------------------------------------------

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,


In [55]:
# Forming the objective Function for the future grid
future_power_res_model.setObjective(quicksum((future_power_demand[i] - future_power_supplied[i, t])
                                              for i in range(future_power_num_nodes) for t in time_periods), GRB.MINIMIZE)

# Solving the power restoration model for the future grid
future_power_res_model.update
future_power_res_model.optimize()

# Extracting the optimal solution for the future grid
future_power_supplied = np.reshape(future_power_res_model.getAttr('x', future_power_supplied).values(),
                            (future_power_num_nodes, num_time_periods))
future_power_avail_mat = np.reshape(future_power_res_model.getAttr('x', future_power_s_links).values(),
                             (future_power_num_links, num_time_periods))
future_power_recovery_times = []
for cnt, val in enumerate(power_failed_links):
    future_power_s_component = future_power_avail_mat[val, power_fail_times[cnt]:]
    future_power_recovery_times.append(next((i for i, x in enumerate(future_power_s_component) if x), None))

# Creating supplied power data frame for the current grid
muns = list(current_power_nodes['Node_Name'].values[62:])
column_names = []
for i in range(num_time_periods):
    column_names.append('Time_Period_' + str(i+1))
current_supply_df = pd.DataFrame({column_names[k]: list(current_power_supplied[62:138, k])
for k in range(num_time_periods)})
current_supply_df.insert(0, 'Municipality', muns.copy())

# Creating supplied power data frame for the future grid
future_supply_df = pd.DataFrame({column_names[k]: list(future_power_supplied[62:138, k])
for k in range(num_time_periods)})
future_supply_df.insert(0, 'Municipality', muns.copy())

# Creating network service level data frame for both current and future grid
current_service_list = np.sum(current_power_supplied, axis=0)/np.sum(current_power_nodes["Demand"].values) * 100
future_service_list = np.sum(future_power_supplied, axis=0)/np.sum(future_power_nodes["Demand"].values) * 100
service_level_df = pd.DataFrame({'Time_Period': list(time_periods), 'Service_Current': current_service_list,
                                 'Service_Future': future_service_list })

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1149550 rows, 261500 columns and 2547770 nonzeros
Model fingerprint: 0xcc551166
Variable types: 257580 continuous, 3920 integer (3920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 25700.000000
Presolve removed 1149072 rows and 260537 columns
Presolve time: 1.26s
Presolved: 478 rows, 963 columns, 2311 nonzeros
Variable types: 855 continuous, 108 integer (108 binary)

Root relaxation: objective 8.264151e+00, 456 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    8.26415    0    6 25700.0000    8.26415   100%     -    1s
H    0     0                      14.2641509    8.26415  42.1%     -    1s
     0     0    8.26415 

# SAID MEJIA

## Understanding the code

In [56]:
np.sum(current_power_supplied, axis=0)

array([ 600.,  600.,  600., 1158., 1665., 1665., 1665., 2570., 2570.,
       2570.])

In [57]:
current_supply_df.loc[(future_supply_df['Municipality'] == 'Rincón') | (future_supply_df['Municipality'] == 'Añasco')]

,Municipality,Time_Period_1,Time_Period_2,Time_Period_3,Time_Period_4,Time_Period_5,Time_Period_6,Time_Period_7,Time_Period_8,Time_Period_9,Time_Period_10
5,Añasco,0.0,0.0,0.0,0.0,0.0,29.849644,0.0,29.849644,29.849644,29.849644
58,Rincón,0.0,0.0,0.0,0.0,0.0,6.704276,0.0,6.704276,6.704276,6.704276


In [58]:
current_power_supplied

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., 19.31198712,
        19.31198712, 19.31198712],
       [ 0.        , 15.46917271, 15.46917271, ..., 15.46917271,
        15.46917271, 15.46917271],
       [ 0.        ,  0.        ,  0.        , ..., 22.12547613,
        22.12547613, 22.12547613]])

In [59]:
df = pd.DataFrame(current_power_supplied)
df

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
133,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.098200,19.098200,19.098200
134,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.670216,39.670216,39.670216
135,0.0,0.000000,0.000000,0.000000,19.311987,0.000000,0.000000,19.311987,19.311987,19.311987
136,0.0,15.469173,15.469173,15.469173,15.469173,15.469173,15.469173,15.469173,15.469173,15.469173


In [60]:
sum(df[9])

2570.0000000000014

In [61]:
sum(df[0])

600.0

In [62]:
current_power_nodes

,Node_Code,Node_Name,Low_Demand,Medium_Demand,High_Demand,Demand,Outage_Low,Outage_Medium,Outage_High,Generation_Capacity,Longitude,Latitude
Node,,,,,,,,,,,,
0,G01,MAYAGUEZ PLANTA,0.000000,0.000000,0.000000,0.000000,12270,33403,84051,200,-67.159520,18.217303
1,G02,CAMBALACHE,0.000000,0.000000,0.000000,0.000000,12270,33403,84051,166,-66.232082,17.948308
2,G03,PALO SECO,0.000000,0.000000,0.000000,0.000000,12270,33403,84051,558,-66.151891,17.942846
3,G04,SJSP,0.000000,0.000000,0.000000,0.000000,12270,33403,84051,600,-66.696612,18.467107
4,G05,AES,0.000000,0.000000,0.000000,0.000000,12270,33403,84051,454,-66.179957,18.467880
...,...,...,...,...,...,...,...,...,...,...,...,...
133,M72,Vega Alta,2.286520,6.103317,10.708362,19.098200,12270,33403,84051,0,-66.337205,18.409490
134,M73,Vega Baja,13.358921,10.795082,15.516212,39.670216,12270,33403,84051,0,-66.397924,18.428475
135,M74,Villalba,8.166763,4.392492,6.752732,19.311987,12270,33403,84051,0,-66.472813,18.128169


In [63]:
sum(current_power_nodes["Demand"])

2570.0000000000005

In [64]:
current_service_list = np.sum(current_power_supplied, axis=0)/np.sum(current_power_nodes["Demand"].values) * 100
current_service_list

array([ 23.3463035 ,  23.3463035 ,  23.3463035 ,  45.05836576,
        64.78599222,  64.78599222,  64.78599222, 100.        ,
       100.        , 100.        ])

## Summarizing the result into a new dataframe

In [107]:
suma_current_grid = pd.DataFrame()
suma_current_grid['Node_Code'] = current_power_nodes['Node_Code']
suma_current_grid['Node_Name'] = current_power_nodes['Node_Name']
suma_current_grid['Demand'] = current_power_nodes['Demand']
suma_current_grid['Time_0'] = df[0]
suma_current_grid['Percentage'] = sumarize_current_grid['Time_0']/sumarize_current_grid['Demand']

In [108]:
suma_current_grid['Percentage'] = sumarize_current_grid['Percentage'].replace(np.nan,0)

In [109]:
suma_current_grid.head()

,Node_Code,Node_Name,Demand,Time_0,Percentage
Node,,,,,
0,G01,MAYAGUEZ PLANTA,0.0,0.0,0.0
1,G02,CAMBALACHE,0.0,0.0,0.0
2,G03,PALO SECO,0.0,0.0,0.0
3,G04,SJSP,0.0,0.0,0.0
4,G05,AES,0.0,0.0,0.0


## Study area

In [110]:
study_area = ['Aguada','Aguadilla','Añasco','Isabela','Hormigueros','Moca','Mayagüez','Rincón']

In [111]:
inde = []
for i in study_area:
    cond = sumarize_current_grid[sumarize_current_grid['Node_Name'] == i]
    inde.append(cond.index[0])

In [112]:
inde

[63, 64, 67, 97, 95, 111, 110, 120]

In [122]:
current_grid_study_area = pd.DataFrame()
current_grid_study_area['Node_Code'] = suma_current_grid.loc[inde,'Node_Code']
current_grid_study_area['Node_Name'] = suma_current_grid.loc[inde,'Node_Name']
current_grid_study_area['Demand'] = suma_current_grid.loc[inde,'Demand']
current_grid_study_area['Time_0'] = suma_current_grid.loc[inde,'Time_0']
current_grid_study_area['Percentage'] = suma_current_grid.loc[inde,'Percentage']

In [123]:
current_grid_study_area

,Node_Code,Node_Name,Demand,Time_0,Percentage
Node,,,,,
63,M02,Aguada,18.208246,0.0,0.0
64,M03,Aguadilla,47.600972,0.0,0.0
67,M06,Añasco,29.849644,0.0,0.0
97,M36,Isabela,21.289861,0.0,0.0
95,M34,Hormigueros,8.513173,0.0,0.0
111,M50,Moca,18.958879,0.0,0.0
110,M49,Mayagüez,76.731217,0.0,0.0
120,M59,Rincón,6.704276,0.0,0.0


In [128]:
# saving the dataframe 
current_grid_study_area.to_csv('Scenarios\current_grid_study_area_time_0.csv') 